In [58]:
import torch
from tqdm import tqdm
import numpy as np

In [7]:
def generate_sample(n=20000, k=4, density = 0.1):
    W = torch.ones((n,1))
    X = torch.rand(n,k)
    mask = torch.rand(n,k) < density
    X[mask] = 0
    theta = torch.rand(k, 1)
    lambdas = torch.exp(X @ theta)
    Y = torch.poisson(lambdas)
    return {
        "n": n,
        "k": k,
        "W": W,
        "X": X.to_sparse(),
        "theta": theta,
        "lambdas": lambdas,
        "Y": Y,
    }

In [8]:
n = 10000
k = 4
density = 0.1
sample = generate_sample(n, k, density)
print(sample["theta"])
print("Is X sparse:", sample["X"].is_sparse)

tensor([[0.0356],
        [0.9555],
        [0.1859],
        [0.5586]])
Is X sparse: True


In [70]:
def train_one_epoch(model, X, Y, W, max_steps=200, optimizer = None, verbose = True):
    if optimizer is None:
        optimizer = torch.optim.RMSprop(model.parameters())
    losses = [torch.nan]
    for i in tqdm(range(max_steps), disable = not verbose):
        optimizer.zero_grad()
        loss = model.get_loss(X, Y, W)
        if abs(losses[-1]-loss)<0.000001:
            if verbose:
                print(f"success after {i} steps")
            break
        losses.append(loss)
        loss.backward()
        optimizer.step()
    return losses

class PoissonRegression(torch.nn.Module):
    def __init__(self, size):
        super(PoissonRegression, self).__init__()
        self.theta = torch.nn.Linear(size, 1)
    
    def forward(self, x):
        linear = self.theta.forward(x)
        return torch.exp(linear)
    
    def get_loss(self, X, Y, W):
        W = W / W.sum()
        weighted_Y = Y * W
        log_likelihood = weighted_Y.T @ self.theta.forward(X) - W.T @ self.forward(X)
        loss = -log_likelihood
        return loss

In [77]:
opt_classes = {
    "torch.optim.SGD" : (torch.optim.SGD, {"lr":0.1}),
    "torch.optim.Adam" : (torch.optim.Adam, {}),
    "torch.optim.RMSprop" : (torch.optim.RMSprop, {}),
    "torch.optim.Adadelta" : (torch.optim.Adadelta, {}),
}
ntrial = 100
results = {key:[] for key in opt_classes.keys()}
for i in tqdm(range(ntrial)):
    for name, (opt_class, params) in opt_classes.items():
        regressor = PoissonRegression(sample["k"])
        optimizer = opt_class(regressor.parameters(), **params)
        losses = train_one_epoch(regressor, sample["X"], sample["Y"], sample["W"], 2000, optimizer, False)
        l2 = torch.norm(sample['theta'] - regressor.theta.weight.T).item()
        results[name].append((len(losses), l2))

100%|█████████████████████████████████████████| 100/100 [32:03<00:00, 19.23s/it]


In [78]:
for key, val in results.items():
    steps = [l[0] for l in val]
    l2 = [l[1] for l in val]
    print(f"{key}: mean steps={np.mean(steps).round(2)}, mean l2={np.mean(l2).round(2)}")

torch.optim.SGD: mean steps=224.75, mean l2=0.04
torch.optim.Adam: mean steps=1915.92, mean l2=0.25
torch.optim.RMSprop: mean steps=207.61, mean l2=0.03
torch.optim.Adadelta: mean steps=411.39, mean l2=0.04


In [51]:
print(regressor.theta.weight)
d = regressor.theta.weight - sample['theta'].T
print(regressor.theta.weight.shape, sample['theta'].T.shape)
print(d)
print(torch.norm(d))

Parameter containing:
tensor([[0.0151, 0.9641, 0.1902, 0.5482]], requires_grad=True)
torch.Size([1, 4]) torch.Size([1, 4])
tensor([[-0.0205,  0.0086,  0.0043, -0.0104]], grad_fn=<SubBackward0>)
tensor(0.0249, grad_fn=<LinalgVectorNormBackward0>)


In [11]:
regressor = PoissonRegression(sample["k"])
losses = train_one_epoch(regressor, sample["X"], sample["Y"], sample["W"], 2000)

  8%|███▏                                   | 166/2000 [00:00<00:07, 232.87it/s]

success after 166 steps


In [43]:
sample["theta"]

tensor([[0.8619],
        [0.4618],
        [0.0853],
        [0.3811]])

In [44]:
regressor.theta.weight

Parameter containing:
tensor([[0.8540, 0.4211, 0.0725, 0.3617]], requires_grad=True)

In [45]:
regressor.theta.weight.grad

In [46]:
def get_baseline_loss(W, Y, X, theta):
    W = W / W.sum()
    weighted_Y = Y * W
    log_likelihood = weighted_Y.T @ X @ theta - W.T @ torch.exp(X @ theta)
    loss = -log_likelihood
    return loss
get_baseline_loss(sample["W"], sample["Y"], sample["X"], sample["theta"])

tensor([[0.2188]])

In [4]:
z

4